In [5]:
library('trendsceek')
library("dplyr")
library('Seurat')
library('data.table')

In [6]:
winsorize <- function(x, n.win = 1){
###Winsorize extreme values
###Ex: Two most extreme values (from each side):
###t(apply(ed, 1, winsorize, 2))

    n.vals = length(x)
    fraction = n.win / n.vals
    if(length(fraction) != 1 || fraction < 0 || fraction > 0.5){
        stop("bad value for 'fraction'")
    }

    win.sorted.ind = order(x)
    x[win.sorted.ind[1:n.win]] = x[win.sorted.ind[n.win + 1]]
    x[win.sorted.ind[(n.vals - n.win + 1):n.vals]] = x[win.sorted.ind[n.vals - n.win]]

    ##fraction-based
    if(0){
        lim = stats::quantile(x, probs = c(fraction, 1 - fraction))

        ##set extreme values
        x[ x < lim[1] ] = lim[1]
        x[ x > lim[2] ] = lim[2]
    }
    
    return(x)
}

In [ ]:
i=43
ff=paste0('Data/field_',i,'_counts_0.csv')
df=read.csv(ff,header = T,row.names=2)
data=df[2:dim(df)[2]]
counts=t(data)
win.counts=t(apply(counts,1,winsorize,4))
Rep<-genefilter_exprmat(counts,min.expr=5,min.ncells.expr = 3)
Rep <-CreateSeuratObject(raw.data = Rep ,min.cells = 3,min.genes = 5 )
Rep_filt<- FilterCells(Rep, subset.names = "nGene", low.thresholds = 500, high.thresholds = Inf)
n.gene=nrow(Rep_filt@raw.data)
n.cell=ncol(Rep_filt@raw.data)
Rep<-NormalizeData(Rep_filt,normalization.method = "LogNormalize",scale.factor = 10000)

info<-read.csv(paste0('Data/field_',i,'_sample_info_0.csv'),row.names=1)
corr<-data.frame(x=info$x,y=info$y)
dim(corr)
pp=pos2pp(corr)
pp=set_marks(pp,win.counts,log.fcn = log10,pseudo.count=1)
sub_pp=head(colnames(pp$marks),10)
#pp2plot
pp2plot=pp_select(pp,sub_pp)

#t0<-Sys.time() 
trendstat_list = trendsceek_test(pp,nrand=10000)
#t1<-Sys.time()

sig_list = extract_sig_genes(trendstat_list, alpha=0.05)
lapply(sig_list, nrow)

sig_gene_v<-sig_list$markvario$gene

sig_gene_c<-sig_list$markcorr$gene

sig_genes=union(sig_gene_v,sig_gene_c)

df_sig=data.frame(field=str(i),n.cell=n.cell,sig_gene=sig_genes)
